In [1]:
!pip install git+https://github.com/neuml/txtai#egg=txtai[pipeline]

  Cloning https://github.com/neuml/txtai to /tmp/pip-install-ncmvcn8r/txtai_2c4e89b301814b29a5f15f39509e8c01
  Running command git clone -q https://github.com/neuml/txtai /tmp/pip-install-ncmvcn8r/txtai_2c4e89b301814b29a5f15f39509e8c01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 4.2 MB 5.4 MB/s 
     |████████████████████████████████| 8.6 MB 42.8 MB/s 
     |████████████████████████████████| 596 kB 57.0 MB/s 
     |████████████████████████████████| 6.6 MB 40.2 MB/s 
     |████████████████████████████████| 4.3 MB 54.3 MB/s 
     |████████████████████████████████| 1.2 MB 63.7 MB/s 
     |████████████████████████████████| 302 kB 74.0 MB/s 
     |████████████████████████████████| 812 kB 57.6 MB/s 
     |████████████████████████████████| 128 kB 65.9 MB/s 
     |████████████████████████████████| 68 kB 7.3 MB/s 
     |████████████████████████████████| 12.8 MB 64.8 MB/s 
  

In [2]:
!pip install jsonlines
import jsonlines
!pip install --no-cache-dir transformers sentencepiece
!pip install langdetect
# from txtai.pipeline import Translation
from langdetect import detect
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import json
import glob
import nltk
nltk.download('punkt')
from copy import deepcopy
from collections import defaultdict
from tqdm import tqdm
from langdetect import detect

from google.colab import drive
drive.mount('/content/drive')

import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Mounted at /content/drive
No GPU available, using the CPU instead.


In [2]:
all_code = {'ar', 'bn', 'ja', 'fi', 'ru', 'ko', 'te'}
translate = Translation()
# translation = translate("This is a test translation into Spanish", "zh")
# translation

# with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_ans_across_lang_nofilter.json") as f:
#   results = json.load(f)

with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_results_100_long_ctxlang_across_lang_nofilter.json") as f:
  results = json.load(f)

In [3]:
len(results)

5001

In [2]:
import torch
print(torch.__version__)

1.11.0+cu113


In [4]:
# {'ctx_lang': 'en',
#    'has_answer': True,
#    'id': '17763030',
#    'score': '39.482452',
#    'text': "resulting genocides and the 1918 influenza pandemic caused another 50 to 100 million deaths worldwide. On 28 June 1914, Gavrilo Princip, a Bosnian Serb Yugoslav nationalist, assassinated the Austro-Hungarian heir Archduke Franz Ferdinand in Sarajevo, leading to the July Crisis. In response, on 23 July Austria-Hungary issued an ultimatum to Serbia. Serbia's reply failed to satisfy the Austrians, and the two moved to a war footing. A network of interlocking alliances enlarged the crisis from a bilateral issue in the Balkans to one involving most of Europe. By July 1914, the great powers of Europe were divided into two coalitions:",
#    'title': 'World War I'},
import gc
from tqdm import tqdm

all_data = []
final_data = []
# for datum in across_lang:
for idx, datum in tqdm(enumerate(results)):
  if idx <= 100*48:
    continue

  # if idx > 100*35:
  #   break

  translated_language = []

  new_datum = []
  src_lang = datum['lang']
  qn = datum['question']
  answer = datum['answers']
  qn_en = translate(qn, 'en')
  ans_en = [translate(ans, 'en') for ans in answer]
  all_context_lang = set([context['ctx_lang'] for context in datum['ctxs'] if context['ctx_lang'] is not None])
  if len(all_context_lang) == 1:
    continue
  for context in datum['ctxs']:
    # if context['has_answer']:
      tar_lang = context['ctx_lang']
      if tar_lang in translated_language:
        continue

      if tar_lang is not None and "zh" in tar_lang:
        tar_lang = "zh"

      sample = {}
      if tar_lang == src_lang:
        for k in datum:
          if k != 'ctxs':
            sample[k] = datum[k]
        sample['context'] = context
        sample['orig_lang'] = src_lang
        sample['orig_qn'] = qn
        new_datum.append(sample)
        translated_language.append(tar_lang)
      # elif tar_lang in all_code: #tar_lang != src_lang
      else:
        try:
          translated_qn = translate(qn_en, tar_lang)
          translated_ans = [translate(ans, tar_lang) for ans in ans_en]
          translated_language.append(tar_lang)
        except:
          print("caught an exception, tar_lang is {}".format(tar_lang))
          continue
        for k in datum:
          if k != 'ctxs':
            sample[k] = datum[k]
        sample['question'] = translated_qn
        sample['answers'] = translated_ans
        sample['lang'] = tar_lang
        sample['context'] = context
        sample['orig_lang'] = src_lang
        sample['orig_qn'] = qn
        new_datum.append(sample)
  all_data.append(new_datum)
  # final_data.append(new_datum)

  break_down = 100
  if idx % break_down == 0 and idx != 0:
    with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_xlingual_samples_top100_part{idx}.json".format(idx=idx//break_down), "w") as outfile:
      json.dump(all_data, outfile)
      print("Wrote to xor_dev_dpr_retrieval_xlingual_samples_nofilter_part{idx}.json".format(idx=idx//break_down))
      all_data = []
      torch.cuda.empty_cache()
      gc.collect()
  # break

# with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_xlingual_samples_nofilter_final.json", "w") as outfile:
#       json.dump(final_data, outfile)
#       print("Wrote to xor_dev_dpr_retrieval_xlingual_samples_nofilter_final.json") 

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:196: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
4822it [06:16, 10.76s/it]

caught an exception, tar_lang is te


4838it [10:19, 33.11s/it]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/283M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/839k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.50M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

5001it [34:03,  2.45it/s]

Wrote to xor_dev_dpr_retrieval_xlingual_samples_nofilter_part50.json


In [ ]:
with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_xlingual_samples_nofilter_part31.json") as f:
  check = json.load(f)
len(check)

100

In [ ]:
with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_xlingual_samples_nofilter_part{idx}.json".format(idx=32), "w") as outfile:
      json.dump(all_data, outfile)

In [ ]:
all_data = []
for idx in range(1, 33):
  with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_xlingual_samples_nofilter_part{idx}.json".format(idx=idx)) as f:
    all_data.extend(json.load(f))

with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_xlingual_samples_nofilter_final.json", "w") as outfile:
  json.dump(all_data, outfile)

In [ ]:
len(all_data)

3124

In [ ]:
all_data[-1]

In [3]:
all_data = []
for idx in range(1, 51):
  with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_xlingual_samples_top100_part{idx}.json".format(idx=idx)) as f:
    all_data.extend(json.load(f))

with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_xlingual_samples_top100_final.json", "w") as outfile:
  json.dump(all_data, outfile)

In [7]:
all_data[-3]

[{'answers': ['1907', '1907', '1907'],
  'context': {'ctx_lang': 'te',
   'has_answer': False,
   'id': '30866226',
   'score': '47.850643',
   'text': 'మద్దుకూరి చంద్రశేఖరరావు తెలుగు సాహిత్యానికి, జర్నలిజానికి మార్క్సిస్టు చూపునిచ్చిన దార్శనికుడు. ఆంధ్ర ప్రాంతంలో కమ్యూనిస్టు ఉద్యమానికి బలమైన పునాదులు వేసినవాడు. ఇతడు కృష్ణాజిల్లా, పెదపారుపూడి మండలం, వెంట్రప్రగడ గ్రామంలో 1907లో జన్మించాడు. ఇతడు విద్యార్థి దశలో జాతీయోద్యమంలో పాల్గొని దాని నుంచి కమ్యూనిస్టు ఉద్యమానికి వచ్చాడు. ఇతడు ఇంజినీరింగ్\u200c విద్యార్థిగా 1930లో ఉప్పుసత్యాగ్రహంలో పాల్గొని జైలుకు వెళ్లాడు. 1932లో వ్యష్టి సత్యాగ్రహంలో పాల్గొన్న సందర్భంలో పోలీసులు ఇతడిని చిత్రహింసలు పెట్టారు. రెండేళ్ల కఠిన జైలు శిక్ష విధించారు. జైలులోనే మార్క్సిస్టు సాహిత్యాన్ని అధ్యయనం చేసి అవగాహన చేసుకున్నాడు. తన పంథా నిర్ధారించుకున్నాడు. తనతోపాటు అనేక మంది యువకులను సమీకరించి పార్టీకి సుశిక్షితులైన సైనికుల్లా మార్చాడు. పుచ్చలపల్లి సుందరయ్య, కంభంపాటి సత్యనారాయణ, చలసాని వాసుదేవరావుతో కలిసి కమ్యూనిస్టు పార్టీ నిర్మాణానికి తనదైన ముద్రతో పనిచేశాడు. క్షేత